# Reconocimiento de Patrones

## Investigación III

### Auto ML: contexto y ejemplos

#### Estudiante: Fabricio Quirós Corella

## 1. Resumen

El presente consiste en un abordaje de los particularidades y una estudio general y superficial de AutoML, cómo se emplea en ML, así como su utilidad en este campo. Igualmente, abarca herramientas que consideran AutoML, como lo son Auto-Sklearn, TPOT y Google Cloud AutoML, donde se ejemplifican mediante el uso de un par de ejemplo prácticos sencillos, para así darle sentido al uso de AutoML dentro del área del Reconocimiento de Patrones.

## 2. AutoML

Es un conjunto de herramientas que automatizan operaciones relativas al proceso general de Machine Learning (ML), tales como el preprocesado, selección del algoritmo, modelado iterativo, ajuste de hiperparámetros y el evaluación del modelo resultante, donde su objetivo consiste en encontrar de manera automática la mejor implementación de ML para un set de datos en particular mediante la evaluación de miles de posibilidades.

### 2.1. Herramientas disponibles

Entre las principales herramientas, cabe mencionar las siguientes, donde, más adelante, se implementan un par ejemplos básicos  basados en dos librerías de Python, cuyo código es abierto, esto para AutoML. A continuación, se enlistan dichos aportes:

* **Auto-Sklearn:** construida sobre *scikit-learn*, capaz de efectuar automáticamente la selección del algoritmo y el ajuste de hiperparámetros, mediante optimización Bayesiana.
* **TPOT (Tree-based Pipeline Optimization):** fundamentada igualmente en la API de *scikit-learn*, la cual genera y optimiza *pipelines* de ML empleando programación genética; bajo este contexto, consiste en que los modelos computados son codificados en "genes", los cuales van evolucionando conforme se aplica el algoritmo en cuestión.
* **Google AutoML:** permite el uso de AutoML para la construcción automática de modelos de aprendizaje a través de las facilidades de ML que ofrece Google. La siguiente publicación de Medium amplia en dicho tema: https://medium.com/@fabisqc0207/google-ml-services-una-breve-rese%C3%B1a-ff095885d8ea.

## 3. Aplicación

El concepto de AutoML resulta de suma importantancia y utilidad en el campo del reconocimiento de patrones, puesto que, en dado caso un algoritmo de ML no funciona lo suficientemente bien o ni siquiera funciona, el proceso de seleccionar y refinar dicho método se convierte en iterativo, donde la construcción de un modelo, a partir de numerosos modelos de ML, empleando una gran cantidad de algoritmos y diversas configuraciones de hiperparámetros, puede ser automatizada, así como la comparación de los resultados correspondientes en términos de rendimiento y precisión, facilitando evidenmente las tareas del científico de datos y las decisiones sobre un modelo de aprendizaje en específico.

## 4. Ejemplo práctico: auto-sklearn

La presente sección incluye la implementación de las tareas de Clasificación y de Regresión, disponibles en la librería de AutoML conocida como ***auto-sklearn***, donde se muestran resultados de los modelos de ML obtenido de forma automática, así como el puntaje de las predicciones respectivas. Inicialmente, se importan librerías útiles, a lo largo de los ejemplos adjuntos.

In [ ]:
# import scikit-learn utilities
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

### 4.1. Clasificación


#### 4.1.1. Crosvalidación.

**1.** Importar las librerías correspondientes.

In [ ]:
# Import auto-sklearn classification
import autosklearn.classification

**2.** Cargar el set de datos disponibles en *scikit-learn* de las imágenes de los dígitos numéricos.

In [ ]:
X, y = sklearn.datasets.load_digits(return_X_y=True)

**3.** Preparar el set de pruebas y de entrenamiento, tanto para la salida (***targets***) y para la entrada (***features***).

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

**4.** Implementar la tarea de AutoML, relativa a la Clasificación, considerando una estrategia de remuestreo de Cros-validación, donde se particiona el set de datos en una cantidad de *folds*, generalmente 5, con el propósito de mejorar su capacidad de generalizar.

In [ ]:
# Create a auto-sklearn classifier object
automl = autosklearn.classification.AutoSklearnClassifier(
    # Time limit in seconds for the search of appropriate models.
    time_left_for_this_task=120,
    # Time limit for a single call to the machine learning model.
    per_run_time_limit=30,
    tmp_folder='/tmp/autoslearn_cv_example_tmp',
    output_folder='/tmp/autosklearn_cv_example_out',
    delete_tmp_folder_after_terminate=True,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)

**5.** Ajustar los modelos de ML a los sets de entrenamiento.

In [ ]:
# fit() changes the data in place, but refit needs the original data. We
# therefore copy the data. In practice, one should reload the data
automl.fit(X_train.copy(), y_train.copy(), dataset_name='digits')

**6.** Aplicar un reajuste al modelo con base en el set de entrenamiento, necesario al emplear Cros-validación.

In [ ]:
# During fit(), models are fit on individual cross-validation folds. To use
# all available data, we call refit() which trains all models in the
# final ensemble on the whole dataset.
automl.refit(X_train.copy(), y_train.copy())

**7.** Desplegar información asociada al proceso de definición del modelo de aprendizaje automático, como el nombre del dataset empleado, así como su métrica y el mejor puntaje de validación, así como la cantidad de corridas existosas y no-exitosas del algoritmo durante el proceso de entrenamiento.

In [ ]:
# Statistics of the training result
print(automl.sprint_statistics())

**8.** Ensamble final encontrado por la librería, correspondiente a los modelos de aprendizaje computados.

In [ ]:
# Return the final model
print(automl.show_models())

**9.** Puntaje de la predicción de las clases a partir del set de pruebas.

In [ ]:
# Use model to predict the classes for a feature set.
predictions = automl.predict(X_test)
print("AutoSklearn Classifier (k-fold CV) - Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

#### 4.1.2. *Holdout*

Técnica de validación que consiste inicialmente en particionar el set de datos en uno de entrenamiento (**TRAIN**), de validación (**VALIDACIÓN**) y pruebas (**TEST**), donde se entrena un modelo y se somete al set de validación, produciendo así una métrica de validación. Nuevamente, se entrena dicho modelo, y posteriormente, este es alimentado con el set de prueba, produciendo en esta ocasión, una métrica de pruebas, favoreciendo a la comparación entre modelos. 

**1.** Ahora, actualizar la tarea de Clasificación de AutoML, empleando esta vez el método de Validación descrito anteriormente.

In [ ]:
# Create a auto-sklearn classifier object
automl = autosklearn.classification.AutoSklearnClassifier(
    # Time limit in seconds for the search of appropriate models.
    time_left_for_this_task=120,
    # Time limit for a single call to the machine learning model.
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_holdout_example_tmp',
    output_folder='/tmp/autosklearn_holdout_example_out',
    delete_tmp_folder_after_terminate=True,
    # 'holdout' with 'train_size'=0.67 is the default argument setting
    # for AutoSklearnClassifier. It is explicitly specified in this example
    # for demonstrational purpose.
    resampling_strategy='holdout',
    resampling_strategy_arguments={'train_size': 0.67}
)

**2.** Ajustar el modelo actualizado al set de entrenamiento nuevamente.

In [ ]:
# Fit the auto-sklearn model on the training data
automl.fit(X_train, y_train, dataset_name='digits')

**3.** Mostrar estadísticas asociadas al proceso de entrenamiento.

In [ ]:
# Statistics of the training result
print(automl.sprint_statistics())

**4.** Desplegar de resultados de la representación final, encontrada por *auto-sklearn*.

In [ ]:
# Return a representation of the final ensemble found by auto-sklearn
print(automl.show_models())

**5.** Predicción final.

In [ ]:
# Use model to predict the classes for a feature set.
predictions = automl.predict(X_test)
print(" AutoSklearn Classifier (Holdout) - Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

### 4.2. Regresión

**1.** Importar la librería de Regresión.

In [ ]:
# Import auto-sklearn regression
import autosklearn.regression

**2.** Una vez importadas las librerías necesarias, se carga el dataset de los precios de las casas de Boston (ejemplo de Regresión: predecir un número continuo, dentro de un rango establecido).

In [ ]:
X, y = sklearn.datasets.load_boston(return_X_y=True)

**3.** Definición de aquella variable que describe el tipo de los atributos empleados.

In [ ]:
feature_types = (['numerical'] * 3) + ['categorical'] + (['numerical'] * 9)

**4.** Nuevamente, preparar tanto el set de pruebas como el de entrenamiento, esto para ***features*** de entrada y ***targets*** de salida.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

**5.** Generar el modelo simple de Regresión.

In [ ]:
# Create a auto-sklearn regressor object
automl = autosklearn.regression.AutoSklearnRegressor(
    # Time limit in seconds for the search of appropriate models.
    time_left_for_this_task=120,
    # Time limit for a single call to the machine learning model.
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
    output_folder='/tmp/autosklearn_regression_example_out',
    delete_tmp_folder_after_terminate=True,
)

**6.** Ajustar el modelo definido al set de entrenamiento, dependiendo del tipo de atributos de entrada.

In [ ]:
# Fit the auto-sklearn model on the training data
automl.fit(X_train, y_train, dataset_name='boston', feat_type=feature_types)

**7.** Resultados asociados al proceso de entrenamiento.

In [ ]:
# Statistics of the training result
print(automl.sprint_statistics())

**8.** Obtener la representación final del ensamble de regresión encontrado por *auto-sklearn*.

In [ ]:
# Return a representation of the final ensemble found by auto-sklearn
print(automl.show_models())

**9.** Predicción final y despliegue del valor asociado al coeficiente de regresión asociado al método considerado por **auto-sklearn**.

In [ ]:
# Use model to predict the classes for a feature set.
predictions = automl.predict(X_test)
print(" AutoSklearn Regressor - R2 score:", sklearn.metrics.r2_score(y_test, predictions))

## 5. Ejemplo práctico: TPOT

Al igual que la sección anterior, se realiza la demostración de ejemplos básicos de Clasificación y Regresión usando la librería de AutoML conocida como **TPOT (Tree-based Pipeline Optimization)**, la cual, al igual que **auto-sklearn**, pretende encontrar automaticamente el modelo que mejor se ajuste al set de datos dado.

### 5.1. Clasificación

**1.** Incluir la librería de Clasificación TPOT.

In [ ]:
# Import the tpot classifier
from tpot import TPOTClassifier

**2.** Cargar nuevamente el set de datos relativos a los dígitos númericos, empleado en los ejemplos anteriores de Clasificación.

In [ ]:
X, y = sklearn.datasets.load_digits(return_X_y=True)

**3.** Ejecutar la división del set de datos en clases a predecir (*targets*) y atributos (*features*) a evaluar, asociados tanto al entrenamiento como a las pruebas.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

**4.** Crear la instancia de la tarea de Clasificación de la librería TPOT.

In [ ]:
# Create a tpot classifier object
automl = TPOTClassifier(
    generations=5,
    # By default, the number of offspring is equal to the number of population size
    population_size=20, offspring_size=None,
    # Recomendation: keep both default parameters unless how the mutation rate affects GP algorithms is understood
    mutation_rate=0.9, crossover_rate=0.1,
    scoring='accuracy', cv=5, # k-fold CrossValidation (k=5)
    # Setting n_jobs=-1 will use as many cores as available on the computer. 
    subsample=1.0, n_jobs=1,
    # Help TPOT from wasting time on evaluating time-consuming pipelines.
    max_time_mins=None, max_eval_time_mins=5,
    # make sure that TPOT will give you the same results each time 
    # you run it against the same data set with that seed
    random_state=None, config_dict=None,
    warm_start=False, memory=None,
    use_dask=False, periodic_checkpoint_folder=None,
    early_stop=None, verbosity=0,
    disable_update_check=False
)

**5.** Ajustar el modelo TPOT al set de entrenamiento.

In [ ]:
# Fit the tpot model on the training data
automl.fit(X_train, y_train)

**6.** Obtener el mejor *pipeline* descubierto por TPOT durante el proceso de optimización, ajustado al set completo de entrenamiento.

In [ ]:
# Show the final model
print(automl.fitted_pipeline_)

**7.** Emplear el modelo optimizado mediante AutoML para realizar predicciones de las clases correspondientes al set de datos de pruebas.

In [ ]:
# Use the optimized pipeline to predict the classes for a feature set.
predictions = automl.predict(X_test)
print("TPOT Regressor - Sci-kit Learn Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

# Optimized pipeline's score on the given testing data (Same result as above)
print("TPOT Regressor - TPOT Accuracy score:", automl.score(X_test, y_test))

### 5.2. Regresión

**1.** Incluir la librería de Regresión TPOT.

In [ ]:
# Import the tpot regressor
from tpot import TPOTRegressor

**2.** Cargar una vez más el *dataset* de los precios de las casas de Boston.

In [ ]:
X, y = sklearn.datasets.load_boston(return_X_y=True)

**3.** Partición del set de datos, tanto para sus atributos como para sus clases, en un conjunto asociado al entrenamiento y otro las pruebas.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

**4.** Crear la instancia de la tarea de Regresión TPOT.

In [ ]:
# Create a tpot object (default parameters are shown)
automl = TPOTRegressor(
    generations=5, 
    population_size=20, offspring_size=None,
    mutation_rate=0.9, crossover_rate=0.1,
    # neg version of mean squared error and related metrics so 
    # TPOT will minimize (instead of maximize) the metric 
    # (scoring default='neg_mean_squared_error')
    scoring='r2', cv=5,
    subsample=1.0, n_jobs=1,
    # Help TPOT from wasting time on evaluating time-consuming pipelines.
    max_time_mins=None, max_eval_time_mins=5,
    # make sure that TPOT will give you the same results each time 
    # you run it against the same data set with that seed
    random_state=None, config_dict=None,
    warm_start=False, memory=None,
    use_dask=False, periodic_checkpoint_folder=None,
    early_stop=None, verbosity=0,
    disable_update_check=False
)

**5.** Ajustar el modelo TPOT computado al *dataset* de entrenamiento.

In [ ]:
# Fit the tpot model on the training data
automl.fit(X_train, y_train)

**6.** Desplegar el *pipeline* óptimo encontrado durante el proceso de optimización, ajustado al set completo de entrenamiento.

In [ ]:
# Show the final model
print(automl.fitted_pipeline_)

**7.** Someter el modelo optimizado mediante AutoML para realizar predicciones de las clases correspondientes al *dataset* de pruebas.

In [ ]:
# Use the optimized pipeline to predict the classes for a feature set.
predictions = automl.predict(X_test)
print("TPOT Regressor - Sci-kit Learn R2 score:", sklearn.metrics.r2_score(y_test, predictions))

# Optimized pipeline's score on the given testing data (Same result as above)
print("TPOT Regressor - TPOT R2 score:", automl.score(X_test, y_test))

## 6. Referencias

* https://automl.github.io/auto-sklearn/stable/#


* https://automl.github.io/auto-sklearn/stable/api.html


* https://epistasislab.github.io/tpot/


* https://github.com/yu-iskw/auto-sklearn-examples


* https://towardsdatascience.com/automated-machine-learning-on-the-cloud-in-python-47cf568859f


* https://www.automl.org/


* https://www.kdnuggets.com/2017/01/current-state-automated-machine-learning.html